In [1]:
from scripts.StreamProcessing import Stream_Data # found in the scripts folder inside StreamProcessing.py. This class uses PySpark

PySpark found
Does Data Folder exist? False


https://berkeleyearth.org/data/

### A note on block 

These might take a while as PySpark attempts to download some additional files

In [ ]:
# Initialize the Stream_Data Class using the topic name/s established in the kafka-config/kafka-config.ipynb and add the host of your kafka instance in the host parameter
emissions_config = Stream_Data(topics='ghg_data', host='localhost:9092')
query, emissions_df = emissions_config.getData()
#temperatures

/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/sqlite-jdbc-3.34.0.jar


23/05/14 10:37:08 WARN Utils: Your hostname, camagakhan-Inspiron-7577 resolves to a loopback address: 127.0.1.1; using 192.168.6.54 instead (on interface wlp60s0)
23/05/14 10:37:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/camagakhan/spark-3.4.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/camagakhan/.ivy2/cache
The jars for the packages stored in: /home/camagakhan/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-07bfb785-cbcb-402f-b3ad-6c5ffc14d4ca;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#com

PySpark found               (0 + 1) / 1][Stage 1:>                  (0 + 1) / 1]
Does Data Folder exist? False
23/05/14 10:37:22 ERROR Executor: Exception in task 0.0 in stage 1.0 (TID 1)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/camagakhan/spark-3.4.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/home/camagakhan/spark-3.4.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 820, in process
    out_iter = func(split_index, iterator)
  File "/home/camagakhan/spark-3.4.0-bin-hadoop3/python/pyspark/sql/streaming/readwriter.py", line 1194, in func_without_process
    f(x)  # type: ignore[operator]
  File "/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/StreamProcessing.py", line 184, in __write__
    cursor.execute(insert_qry, values)
sqlite3.IntegrityError: NOT NULL constraint failed: ghg_data.PowerCodeCode

	at or

-------------------------------------------
Batch: 0
-------------------------------------------
+------+---------+---+----------------+-----+--------------------+----+----+--------+--------------------+-------------+---------+-------------------+---------------+----------+---------+-----+
|﻿"COU"|  Country|POL|       Pollutant|  VAR|            Variable| YEA|Year|UnitCode|                Unit|PowerCodeCode|PowerCode|ReferencePeriodCode|ReferencePeriod|     Value|FlagCodes|Flags|
+------+---------+---+----------------+-----+--------------------+----+----+--------+--------------------+-------------+---------+-------------------+---------------+----------+---------+-----+
|   AUS|Australia|GHG|Greenhouse gases|TOTAL|Total  emissions ...|1990|1990|    null|Tonnes of CO2 equ...|         null|Thousands|               null|           null|425624.307|     null|     |
|   AUS|Australia|GHG|Greenhouse gases|TOTAL|Total  emissions ...|1991|1991|    null|Tonnes of CO2 equ...|         null|Thousan

In [ ]:
# Initialize the Stream_Data Class using the topic name/s established in the kafka-config/kafka-config.ipynb and add the host of your kafka instance in the host parameter
temperature_config = Stream_Data(topics='temperature', host='localhost:9092')
query_temperature, temperature_df = temperature_config.getData()

### NOTE
Everything is configered while initializing the PySpark Session in the ```StreamProcessing.py``` class. DO NOT remove the ```from graphframes import *```, as the jar files are loaded while getting the data from the previous code block

In [ ]:
from graphframes import * 